In [ ]:
!/usr/local/cuda/bin/nvcc --version
!pip install git+https://github.com/andreinechaev/nvcc4jupyter.git
%load_ext nvcc_plugin
!cuda-install-samples-11.2.sh ~ && cd /root/NVIDIA_CUDA-11.2_Samples

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2021 NVIDIA Corporation
Built on Sun_Feb_14_21:12:58_PST_2021
Cuda compilation tools, release 11.2, V11.2.152
Build cuda_11.2.r11.2/compiler.29618528_0
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/andreinechaev/nvcc4jupyter.git to /tmp/pip-req-build-0ffas12n
  Running command git clone -q https://github.com/andreinechaev/nvcc4jupyter.git /tmp/pip-req-build-0ffas12n
  Created wheel for NVCCPlugin: filename=NVCCPlugin-0.0.2-py3-none-any.whl size=4307 sha256=12816fa0f1f33d1365b36b91b7a66e11c02c4afc3be392f690070b254435b9d1
  Stored in directory: /tmp/pip-ephem-wheel-cache-hlw4somc/wheels/ca/33/8d/3c86eb85e97d2b6169d95c6e8f2c297fdec60db6e84cb56f5e
Successfully built NVCCPlugin
created output directory at /content/src
Out bin /content/result.out
Copying samples to /root/NVIDIA_CUDA-11.2_Samples now...
Finished copying samples.


In [ ]:
%%cuda --name curand.cu
//Пример 1. Код main - функции
#include <cublas_v2.h>
#include <malloc.h>
#include <stdio.h>
#include <stdlib.h>

//Пример 2. Код функции ядра
__global__ void addKernel(int* c, int* a, int* b, unsigned int size)
{
    int gridSize = blockDim.x * gridDim.x;
        int index = blockIdx.x * blockDim.x + threadIdx.x; //threadIdx.x; // 
        for (; index < size; index += gridSize) {
            if (index < size) {
                c[index] = a[index] + b[index]; // Вычислим id нити, в которой происходит суммирование с помощью формулы
                
            }
        }
}

#define kernel addKernel

/*
int, 2 400 000 (/ 2 / 4), 
[(4096, 1024);(2048, 1024);(4096, 256);(2048, 256);(4096, 64);(2048, 64)]
тип, исходная размерность массива (делители для получения остальных размерностей), 
[перечень пар (GridDim,BlockDim)]
*/
int main(int argc, char* argv[])
{

    int n = 600000; //количество элементов в суммируемых массивах
    int GRID_SIZE = 4096;
    int BLOCK_SIZE = 1024;

    // Size, in bytes, of each vector
    int n2b = n * sizeof(int);

    //BLOCK_SIZE - количество нитей в блоке кратное 32
    int n2 = n;
        //BLOCK_NUMBER * BLOCK_SIZE * sizeof(int);;
    

    // Выделение памяти на хосте
    int* a = (int*)calloc(n2, sizeof(int));
    int* b = (int*)calloc(n2, sizeof(int));
    int* c = (int*)calloc(n2, sizeof(int));
    // Инициализация массивов
    for (int i = 0; i < n; i++) {
        a[i] = 1;
        b[i] = 1;
        
    }
    // Выделение памяти на устройстве
    int* adev = NULL;
    cudaError_t cuerr = cudaMalloc((void**)&adev, n2b);
    if (cuerr != cudaSuccess)
    {
        fprintf(stderr, "Cannot allocate device array for a: %s\n",
            cudaGetErrorString(cuerr));
        return 0;
    }

    int* bdev = NULL;
    cuerr = cudaMalloc((void**)&bdev, n2b);
    if (cuerr != cudaSuccess)
    {
        fprintf(stderr, "Cannot allocate device array for b: %s\n",
            cudaGetErrorString(cuerr));
        return 0;
    }

    int* cdev = NULL;
    cuerr = cudaMalloc((void**)&cdev, n2b);
    if (cuerr != cudaSuccess)
    {
        fprintf(stderr, "Cannot allocate device array for c: %s\n",
            cudaGetErrorString(cuerr));
        return 0;
    }

    // Создание обработчиков событий
    cudaEvent_t start, stop;
    float gpuTime = 0.0f;
    cuerr = cudaEventCreate(&start);
    if (cuerr != cudaSuccess)
    {
        fprintf(stderr, "Cannot create CUDA start event: %s\n",
            cudaGetErrorString(cuerr));
        return 0;
    }

    cuerr = cudaEventCreate(&stop);
    if (cuerr != cudaSuccess)
    {
        fprintf(stderr, "Cannot create CUDA end event: %s\n",
            cudaGetErrorString(cuerr));
        return 0;
    }

    // Копирование данных с хоста на девайс
    cuerr = cudaMemcpy(adev, a, n2b, cudaMemcpyHostToDevice);
    if (cuerr != cudaSuccess)
    {
        fprintf(stderr, "Cannot copy a array from host to device: %s\n",
            cudaGetErrorString(cuerr));
        return 0;
    }

    cuerr = cudaMemcpy(bdev, b, n2b, cudaMemcpyHostToDevice);
    if (cuerr != cudaSuccess)
    {
        fprintf(stderr, "Cannot copy b array from host to device: %s\n",
            cudaGetErrorString(cuerr));
        return 0;
    }

    // Установка точки старта
    cuerr = cudaEventRecord(start, 0);
    if (cuerr != cudaSuccess)
    {
        fprintf(stderr, "Cannot record CUDA event: %s\n",
            cudaGetErrorString(cuerr));
        return 0;
    }

    //Запуск ядра
    kernel << < GRID_SIZE, BLOCK_SIZE >> > (cdev, adev, bdev, n);

    cuerr = cudaGetLastError();
    if (cuerr != cudaSuccess)
    {
        fprintf(stderr, "Cannot launch CUDA kernel: %s\n",
            cudaGetErrorString(cuerr));
        return 0;
    }

    // Синхронизация устройств
    cuerr = cudaDeviceSynchronize();
    if (cuerr != cudaSuccess)
    {
        fprintf(stderr, "Cannot synchronize CUDA kernel: %s\n",
            cudaGetErrorString(cuerr));
        return 0;
    }

    // Установка точки окончания
    cuerr = cudaEventRecord(stop, 0);
    if (cuerr != cudaSuccess)
    {
        fprintf(stderr, "Cannot copy c array from device to host: %s\n",
            cudaGetErrorString(cuerr));
        return 0;
    }

    // Копирование результата на хост
    cuerr = cudaMemcpy(c, cdev, n2b, cudaMemcpyDeviceToHost);
    if (cuerr != cudaSuccess)
    {
        fprintf(stderr, "Cannot copy c array from device to host: %s\n",
            cudaGetErrorString(cuerr));
        return 0;
    }

    // Расчет времени
    cuerr = cudaEventElapsedTime(&gpuTime, start, stop);
    printf("time spent executing %s: %.9f seconds\n", "kernel", gpuTime / 1000);

    // Очищение памяти
    cudaEventDestroy(start);
    cudaEventDestroy(stop);
    cudaFree(adev);
    cudaFree(bdev);
    cudaFree(cdev);

    free(a);
    free(b);
    free(c);


    return 0;
}


'File written in /content/src/curand.cu'

In [ ]:
!nvcc -o /content/src/curand /content/src/curand.cu -lcurand -lcublas


In [ ]:
!/content/src/curand

time spent executing kernel: 0.000058400 seconds


In [ ]:
%%cuda --name consistently.cu
#include <math.h>
#include <stdio.h>
#include <stdlib.h>
#include <iostream>
#include <chrono>
using std::chrono::duration;
using std::chrono::duration_cast;
using std::chrono::high_resolution_clock;

//int, 2 400 000 (+7), [2, 4, 8], 28
int main()
{
	int T = 12, Q = 1;
	double timeCount = 0;
	int NMAX = 600000;

	for (int t = 0; t < 12; t++) {

		int* a = new int[NMAX];
		int* sum = new int[NMAX];
		int* b = new int[NMAX];

		for (int i = 0; i < NMAX; i++) {
			a[i] = 1;
			b[i] = 1;
		}

		

		const auto startTime = high_resolution_clock::now();
		//for (int n = 0; n < N;n++) {
			
			for (int i = 0; i < NMAX; i++) {
				for (int q = 0; q < Q; q++) {
					sum[i] = a[i] + b[i];
				}
			}
		//}
		const auto endTime = high_resolution_clock::now();
		std::chrono::duration<double> diff = endTime - startTime;
		timeCount+=diff.count();
    delete[] a;
		delete[] b;
		delete[] sum;
  }
    std::cout << "Time " << timeCount/T << " s\n";

		
	//}

	return 0;
}

'File written in /content/src/consistently.cu'

In [ ]:
!nvcc -o /content/src/consistently /content/src/consistently.cu -lcurand -lcublas

In [ ]:
!/content/src/consistently

Time 0.00215753 s
